In [14]:
import pyrealsense2 as rs
import numpy as np
import cv2
import random
import torch
import time
from ultralytics import YOLO

In [15]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.148  Python-3.9.16 torch-2.0.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
Setup complete  (8 CPUs, 15.8 GB RAM, 71.5/97.7 GB disk)


In [16]:
# Load the YOLOv8 model
model = YOLO('best.pt')

In [17]:
def get_mid_pos(frame,box,depth_data,randnum,color):
    distance_list = []
    mid_pos = [(box[0] + box[2])//2, (box[1] + box[3])//2] #确定索引深度的中心像素位置
    ratio = 0.7
    for i in range(randnum):
        len = 16                           #the length of the simple line      
        bias1 = (-len/2+1)+len*i/randnum   # set simple points from -len/2+1 to len/2+1       
#         dist = depth_data[int(mid_pos[1]*ratio + 102), int(mid_pos[0]*ratio + 164 + bias1)]  # *1.3 is to meet the mismatch between the GRB and depth image.
        dist = depth_data[int(mid_pos[1]*ratio), int(mid_pos[0]*ratio + bias1)]
        cv2.circle(frame, (int(mid_pos[0] + bias1), int(mid_pos[1]) ), 2, color, -1)
        if dist:  
            distance_list.append(dist)
        dist = depth_data[int(mid_pos[1]*ratio+ 102 + bias1), int(mid_pos[0]*ratio + 164)]
        cv2.circle(frame, (int(mid_pos[0]), int(mid_pos[1] + bias1)), 2, color, -1)
        if dist:  
            distance_list.append(dist)

    distance_list = np.array(distance_list)
    distance_list = np.sort(distance_list)[randnum//2-randnum//4:randnum//2+randnum//4] #冒泡排序+中值滤波
    return np.mean(distance_list)

In [18]:
def dectshow(org_img, box,depth_data, color):
    # img = org_img.copy()
    # cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
    dist = get_mid_pos(org_img, box, depth_data, 16, color)
    cv2.putText(org_img, str(dist / 10)[:4] + 'cm',  # https://blog.csdn.net/weixin_42039090/article/details/80679935/
                (int(box[0]), int(box[3])+18), cv2.FONT_ITALIC, 0.7, color, 2)  #(55, 20, 200)
    # cv2.imshow('dec_img', img)
    return org_img

In [19]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
# Start streaming
profile = pipeline.start(config)
device = profile.get_device()
device.hardware_reset()
time.sleep(1)

# Loop through the video frames
while True:
    # Read a frame from the video
    # success, frame = cap.read()
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    if not depth_frame or not color_frame:
        continue
    # Convert images to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Run YOLOv8 inference on the frame
    # results = model.track(color_image,conf=0.65,show=False)
    results = model.track(source=color_image, conf=0.65, iou=0.5, show=False, device="CPU")
    frame = results[0].plot()
    print(results)
    # frame_distance = dectshow(frame, box, depth_image)

    # # Visualize the results on the frame
    # annotated_frame = results[0].plot()

    # # Display the annotated frame
    target = results[0].boxes
    targetlabel = target.cls.cpu().numpy() #.cpu().numpy()  is to change the format from tensor to numpy which is more common
    if targetlabel.size > 0:
        boxes= results[0].boxes
        box=target[0].xyxy[0]
    else:
        frame_distance = frame
         
    # targetpos = target.xywh.cpu().numpy()
    # targetconf = target.conf
    for i in range(targetlabel.size):
        if targetlabel[i] == 0:   # To judege if the target is BOX(it is set as num 0 in jw0413.yaml when train the model)  
            # target_x = targetpos[i][0]
            # target_y = targetpos[i][1]
            # cv2.circle(frame,(int(target_x),int(target_y)),3,(255,0,0),-1)   # Plot a point at the certer of the target in the picture.            
            # print(str(i)+".Box is at ("+str(target_x)+","+str(target_y)+")")
            frame_distance = dectshow(frame, boxes[i].xyxy[0], depth_image,(255,0,0))

        elif targetlabel[i] == 1:
            # target_x = targetpos[i][0]
            # target_y = targetpos[i][1]
            # cv2.circle(annotated_frame,(int(target_x),int(target_y)),3,(0,0,255),-1)   
            # print(str(i)+".Earth is at ("+str(target_x)+","+str(target_y)+")")
            frame_distance = dectshow(frame, boxes[i].xyxy[0], depth_image,(0,0,255))
        elif targetlabel[i] == 2:
            # target_x = targetpos[i][0]
            # target_y = targetpos[i][1]
            # cv2.circle(annotated_frame,(int(target_x),int(target_y)),3,(0,255,0),-1)               
            # print(str(i)+".Saturn is at ("+str(target_x)+","+str(target_y)+")")    
            frame_distance = dectshow(frame, boxes[i].xyxy[0], depth_image,(0,255,0))             
        else:
            print("Invalid target label, please add the new item to the target classes")
    # cv2.imshow("TRANSCENT Target Reconginazition", annotated_frame) # Display the annotated frame 
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
    # Stack both images horizontally
    images = np.hstack((frame_distance, depth_colormap))

    cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RealSense', images)    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
# Release the video capture object and close the display window
pipeline.stop() # cap.release()
cv2.destroyAllWindows()


0: 480x640 1 toyball, 184.2ms
Speed: 7.0ms preprocess, 184.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'toyball', 1: 'toyblock'}
orig_img: array([[[33, 45, 74],
        [36, 48, 77],
        [36, 49, 75],
        ...,
        [13, 29, 41],
        [13, 32, 41],
        [13, 32, 41]],

       [[33, 47, 76],
        [34, 48, 77],
        [32, 49, 77],
        ...,
        [17, 30, 43],
        [16, 33, 42],
        [15, 32, 41]],

       [[34, 51, 79],
        [34, 51, 79],
        [34, 51, 79],
        ...,
        [17, 30, 43],
        [17, 34, 43],
        [16, 33, 42]],

       ...,

       [[ 0,  8, 18],
        [ 0, 11, 21],
        [ 0, 10, 23],
        ...,
        [ 0, 13, 34],
        [ 0, 13, 34],
        [ 0, 13, 34]],

       [[ 0,  8, 23],
        [ 0,  9, 24],
        [ 0,  8, 23],
        ...,
        [ 0, 15, 36],
        [ 0, 13, 34],
        [ 0, 12, 33]],

       [[ 0,  7, 25],
        [ 0,  8, 26],
        [

0: 480x640 (no detections), 203.6ms
Speed: 2.0ms preprocess, 203.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'toyball', 1: 'toyblock'}
orig_img: array([[[193, 186, 176],
        [193, 186, 176],
        [195, 188, 178],
        ...,
        [143, 137, 125],
        [143, 137, 125],
        [143, 137, 125]],

       [[190, 185, 175],
        [188, 183, 173],
        [193, 189, 177],
        ...,
        [144, 141, 131],
        [143, 141, 129],
        [142, 140, 128]],

       [[186, 184, 172],
        [184, 182, 170],
        [186, 184, 172],
        ...,
        [141, 141, 130],
        [143, 140, 130],
        [144, 141, 131]],

       ...,

       [[ 38,  75,  97],
        [ 40,  77,  99],
        [ 40,  77,  99],
        ...,
        [ 72,  98, 125],
        [ 65,  94, 120],
        [ 65,  94, 120]],

       [[ 40,  79, 101],
        [ 38,  77,  99],
        [ 39,  78, 100],
        ...,
        [ 73, 100, 124],
        [ 67,  

0: 480x640 (no detections), 221.2ms
Speed: 2.4ms preprocess, 221.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



...

0: 480x640 1 toyball, 268.1ms
Speed: 5.5ms preprocess, 268.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)



[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'toyball', 1: 'toyblock'}
orig_img: array([[[195, 191, 179],
        [195, 191, 179],
        [196, 191, 181],
        ...,
        [154, 143, 137],
        [156, 145, 139],
        [154, 143, 137]],

       [[193, 191, 179],
        [194, 192, 180],
        [200, 198, 186],
        ...,
        [152, 147, 137],
        [154, 147, 137],
        [152, 145, 135]],

       [[195, 193, 181],
        [194, 192, 180],
        [199, 197, 183],
        ...,
        [148, 145, 135],
        [150, 145, 135],
        [150, 145, 135]],

       ...,

       [[ 49,  81, 104],
        [ 47,  79, 102],
        [ 45,  80, 102],
        ...,
        [ 71,  97, 124],
        [ 73,  97, 124],
        [ 72,  96, 123]],

       [[ 50,  80, 103],
        [ 50,  80, 103],
        [ 45,  80, 102],
        ...,
        [ 70,  99, 125],
        [ 69,  

0: 480x640 1 toyball, 268.5ms
Speed: 4.6ms preprocess, 268.5ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)



[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'toyball', 1: 'toyblock'}
orig_img: array([[[198, 191, 181],
        [199, 192, 182],
        [200, 196, 184],
        ...,
        [153, 148, 138],
        [153, 148, 138],
        [152, 147, 137]],

       [[193, 194, 181],
        [194, 195, 182],
        [194, 195, 182],
        ...,
        [153, 148, 138],
        [152, 149, 139],
        [148, 145, 135]],

       [[190, 193, 180],
        [192, 195, 182],
        [194, 197, 184],
        ...,
        [146, 143, 133],
        [148, 145, 135],
        [148, 145, 135]],

       ...,

       [[ 42,  81,  98],
        [ 43,  82,  99],
        [ 45,  84, 101],
        ...,
        [ 72, 104, 127],
        [ 69, 101, 124],
        [ 66,  98, 121]],

       [[ 47,  84, 101],
        [ 46,  83, 100],
        [ 46,  83, 100],
        ...,
        [ 71, 101, 124],
        [ 70, 1

KeyboardInterrupt: 